In [2]:
import glob

In [3]:
json_files = glob.glob("../data/training/*.json")

In [5]:
num_inputs = 10

In [9]:
train = json_files[0: int(num_inputs*0.7) ]
test = json_files[int(num_inputs*0.7): num_inputs]

In [10]:
train

['../data/training/AK_Valdez.json',
 '../data/training/NV_Tonopah_321370_1961_62500_geo_mosaic.json',
 '../data/training/CO_Silverton_451014_1955_24000_geo_mosaic.json',
 '../data/training/WI_StevensPoint_700274_1956_48000_geo_mosaic.json',
 '../data/training/ID_Bellevue_238908_1957_62500_geo_mosaic.json',
 '../data/training/WI_Wyeville_503636_1958_48000_geo_mosaic.json',
 '../data/training/CO_GrandJunction.json']

In [11]:
test

['../data/training/IN_ClayCity_160547_1908_48000_geo_mosaic.json',
 '../data/training/NV_SanAntonioRanch_321264_1964_62500_geo_mosaic.json',
 '../data/training/NM_TetillaPeak.json']

In [12]:
json_files[0:10]

['../data/training/AK_Valdez.json',
 '../data/training/NV_Tonopah_321370_1961_62500_geo_mosaic.json',
 '../data/training/CO_Silverton_451014_1955_24000_geo_mosaic.json',
 '../data/training/WI_StevensPoint_700274_1956_48000_geo_mosaic.json',
 '../data/training/ID_Bellevue_238908_1957_62500_geo_mosaic.json',
 '../data/training/WI_Wyeville_503636_1958_48000_geo_mosaic.json',
 '../data/training/CO_GrandJunction.json',
 '../data/training/IN_ClayCity_160547_1908_48000_geo_mosaic.json',
 '../data/training/NV_SanAntonioRanch_321264_1964_62500_geo_mosaic.json',
 '../data/training/NM_TetillaPeak.json']

In [13]:
import json

In [27]:
inputs = []
for json_file in json_files:
    with open(json_file) as jfile:
        label_info = json.load(jfile)

    img_ht = label_info["imageHeight"]
    img_wd = label_info["imageWidth"]

    inp_file = json_file.replace(".json", ".tif")
    #print(inp_file)

    for shape in label_info["shapes"]:
        label = shape["label"]
        label_type = label.split("_")[-1]
        label_file = inp_file.replace(".tif", "")+"_"+label+".tif"
        points = shape["points"]
        inputs.append( [json_file, img_ht, img_wd, inp_file, label, label_type, label_file, points] )
    

df = pd.DataFrame(inputs, columns=["json_file", "height", "width", "inp_file", "label", "label_type", "label_file", "points"])



In [28]:
df.label_type.value_counts()

poly    3672
pt       393
line     197
Name: label_type, dtype: int64

In [29]:
poly_files = df.loc[df.label_type == "poly"]

In [ ]:
poly_files

In [39]:
poly_files.to_csv("poly_input_info.csv",index=False)

In [32]:
pt_files = df.loc[df.label_type == "pt"]

In [40]:
pt_files.to_csv("pt_input_info.csv",index=False)

In [34]:
line_files = df.loc[df.label_type == "line"]

In [41]:
line_files.to_csv("line_input_info.csv",index=False)

In [42]:
df.to_csv("all_input_info.csv",index=False)

In [37]:
short = pd.concat([ poly_files.iloc[0:10], line_files.iloc[0:3], pt_files.iloc[0:2] ])

In [ ]:
short

In [43]:
short.to_csv("short_input_info.csv",index=False)

In [45]:
import glob
import pandas as pd 
import json 
import os 
def prepare_input_info_csv(input_dir, info_file="input_info.csv"):
    json_files = glob.glob(os.path.join(input_dir, "*.json"))
    inputs = []
    for json_file in json_files:
        with open(json_file) as jfile:
            label_info = json.load(jfile)

        img_ht = label_info["imageHeight"]
        img_wd = label_info["imageWidth"]

        inp_file = json_file.replace(".json", ".tif")
        #print(inp_file)

        for shape in label_info["shapes"]:
            label = shape["label"]
            label_type = label.split("_")[-1]
            label_file = inp_file.replace(".tif", "")+"_"+label+".tif"
            points = shape["points"]
            inputs.append( [json_file, img_ht, img_wd, inp_file, label, label_type, label_file, points] )
        

    df = pd.DataFrame(inputs, columns=["json_file", "height", "width", "inp_file", "label", "label_type", "label_file", "points"])
    df.to_csv(info_file, index=False)



In [ ]:
prepare_input_info_csv("../data/training/", "inp_info.csv")

In [2]:
import os
import pandas as pd

In [3]:
sdf = pd.read_csv("../eda/short_input_info.csv")

In [6]:
copy_path = "../data/short_inp/"
if not os.path.isdir(copy_path):
    os.mkdir("../data/short_inp/")

In [4]:
import shutil
# lets first copy all input files
in_files = set(sdf.inp_file)

In [7]:
for inf in in_files:
    shutil.copy(inf, os.path.join(copy_path, os.path.basename(inf)))

In [8]:
# lets copy the json files
json_files = set(sdf.json_file)
for inf in json_files:
    shutil.copy(inf, os.path.join(copy_path, os.path.basename(inf)))

# lets copy label files
label_files = set(sdf.label_file)
for inf in label_files:
    shutil.copy(inf, os.path.join(copy_path, os.path.basename(inf)))

In [9]:
tiled_input_dir = "../data/short_inp/tiled_inputs"
tiled_label_dir = "../data/short_inp/tiled_labels"
tiled_mask_dir = "../data/short_inp/tiled_masks"

In [1]:
# tile the inputs, labels, masks

import img2tiles

In [10]:
tile_size = 256
for inf in in_files:
    in_tiles = img2tiles.split_image_into_tiles(inf, tiled_input_dir, tile_size)

In [11]:
for inf in label_files:
    label_mask_tiles = img2tiles.split_image_into_tiles(inf, tiled_mask_dir, tile_size)

In [20]:
import ast
inputs_descriptor = []
for idx, row in sdf.iterrows():
    in_tiles = img2tiles.split_image_into_tiles(row.inp_file, tiled_input_dir, tile_size)

    points = ast.literal_eval(row.points)
    label_mask_tiles = img2tiles.split_image_into_tiles(row.label_file, tiled_mask_dir, tile_size)
    label_pattern_fname = img2tiles.make_label_pattern(row.inp_file, row.label, points, tiled_label_dir, tile_size)

    assert(len(label_mask_tiles) == len(in_tiles))
    for tile_no, tile in enumerate(label_mask_tiles):
        empty_tile = img2tiles.check_non_zero_tile(os.path.join(tiled_mask_dir, label_mask_tiles[tile_no]))
        inputs_descriptor.append([row.inp_file, row.height, row.width, in_tiles[tile_no], label_pattern_fname, label_mask_tiles[tile_no], empty_tile, tile_size])

In [21]:
df = pd.DataFrame(inputs_descriptor, columns = ["orig_file", "orig_ht", "orig_wd", "tile_inp", "tile_legend", "tile_mask", "empty_tile", "tile_size"])
df.to_csv("short_input.csv", index=False)

In [22]:
import prepare

In [23]:
prepare.prepare_balanced_inputs("short_input.csv", "short_train.csv", "short_test.csv")

avg_true of the dataset = 0.016696022310289153
length of true = 455
length of true_train = 364
length of true_test = 91
0.027826703850481922
Avg_True  0.37877211238293446
Avg_True test 0.37916666666666665
train len: 961, test len: 240
